In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from tqdm import tqdm
import uuid

from datasets import Dataset

/home/kosenko/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
thread = open("./verbalist/datasets/bugurt_thread/messages40.html").read()

soup = BeautifulSoup(thread)
all_messages = soup.find_all("div", class_="message")

In [3]:
len(all_messages)

1068

In [15]:
import markdownify

text = str(all_messages[100].find("div", class_="text"))
# text
text = str(
    markdownify.markdownify(
        text,
    )
)
text = text.replace("предложено через", "")
text = text.replace("[@tgbtbot](https://t.me/tgbtbot)", "")
text = text.replace("[БУГУРТ-ТРЕД](https://t.me/bugurtthread)", "")
print(text)


СЫЧ ШАРИТ ЗА ВСЁ ЧТО ТОЛЬКО МОЖНО  
@  
ЗНАЕТ АНГЛИЙСКИЙ/РУССКИЙ/ТАТАРСКИЙ/КАЗАХСКИЙ/ЭЛЬФИЙСКИЙ ЯЗЫКИ И ПРИ ВИДЕ КАКОЙ ЛИБО ОШИБКИ В ЧУЖОМ ТЕКСТЕ НАЧИНАЕТ МИРОВУЮ ВОЙНУ  
@  
УМЕЕТ ВЫПОЛНЯТЬ ЗАДАЧКИ ИЗ ВЫСШЕЙ МАТЕМАТИКИ, В ШКОЛЕ ДИКО РОФЛИЛ С БЫДЛАНОВ ЕРОХ ТУПИВШИХ У ДОСКИ  
@  
ХОРОШО РАЗБИРАЕТСЯ В ПОЛИТИКЕ И УПРАВЛЕНИЕМ ГОСУДАРСТВА, ПОТОМУЧТО КАЖДЫЙ ДЕНЬ ИГРАЕТ В ХОЙКУ/ЦИВКУ И СМОТРИТ ПОЛИТИЧЕСКИЕ/ИСТОРИЧЕСКИЕ КАНАЛЫ  
@  
МИЛИТАРИСТ, ОЧЕНЬ СИЛЬНО ЗАИНТЕРЕСОВАН ОРУЖИЕМ И НЕ ПРОПУСКАЕТ НИОДНОГО ВИДЕО "РАЗРУШИТЕЛЬНОЕ РАНЧО (В ПЕРЕВОДЕ ZEBRA)"  
@  
ХОРОШО РАЗБИРАЕТСЯ В КАЧЕСТВЕНЫХ ФИЛЬМАХ/ИГРАХ/КНИГАХ/МУЗЫКЕ, ЕГО ВКУСЫ ОЧЕНЬ СИЛЬНО ВЫДЕЛЯЮТСЯ СРЕДИ СЕРОЙ МАССЫ БЫДЛАНОВ  
@  
ВСЕГДА ВЫИГРЫВАЕТ ВО ВСЕХ СПОРАХ В ИНТЕРНЕТЕ, УМЕЕТ ТРОЛИТЬ ВАТНОЕ БЫДЛО, ПОДЖИГАЕТ АНУС МОРАЛФАГАМ  
@  
НО ПРИ ЭТОМ ВСЁМ НЕ МОЖЕТ В ОБЩЕНИЕ/ОТНОШЕНИЯ С ДЕВУШКАМИ  
  




In [16]:
text.count("@")

7

In [46]:
import re

dataset = []

for num in range(40, 45):
    thread = open(f"./verbalist/datasets/bugurt_thread/messages{num}.html").read()

    soup = BeautifulSoup(thread)
    all_messages = soup.find_all("div", class_="message")
    for i in range(len(all_messages)):
        text = str(all_messages[i].find("div", class_="text"))
        # text
        text = str(
            markdownify.markdownify(
                text,
            )
        )
        text = text.replace("предложено через", "")
        text = text.replace("предложено  через", "")
        text = text.replace("[@tgbtbot](https://t.me/tgbtbot)", "")
        text = text.replace("[БУГУРТ-ТРЕД](https://t.me/bugurtthread)", "")
        text = re.sub(r"\[(.*?)\]\(.*?\)", "", text).strip()
        if "@" in text:
            if text.count("@") >= 3:
                # print(text)
                dataset.append({"bugurt": text})
                # print("=" * 100)

In [24]:
len(dataset)

2521

In [47]:
Dataset.from_list(dataset).push_to_hub("dim/raw_bugurts_2.5k")

Pushing dataset shards to the dataset hub: 100%|██████████| 1/1 [00:00<00:00, 18641.35it/s]
Updating downloaded metadata with the new split.


In [44]:
dataset = Dataset.from_list(dataset)

In [45]:
dataset.to_csv("./verbalist/datasets/bugurt_thread/bugurt_thread.csv")

Creating CSV from Arrow format: 100%|██████████| 3/3 [00:00<00:00, 48.52ba/s]


2831798

### sumarize bugurts


In [28]:
import openai
import pandas as pd
import numpy as np
import time

openai.api_key = open("./chat_gpt_token").read()


def chat_with_chatgpt(prompt, model="gpt-3.5-turbo"):
    chat_completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.0,
    )

    return chat_completion["choices"][0]["message"]["content"].strip()


user_prompt = "Hello world."
chatbot_response = chat_with_chatgpt(user_prompt)
print(chatbot_response)

Hello! How can I assist you today?


In [43]:
text = dataset[10]["bugurt"]

prompt = f"""
Текст: {text}

Напиши бугурт на основе текста по типу:
Напиши бугурт на тему...
или
Напиши бугурт про то как...
или
Напиши бугурт о том как...

Бугурт должен быть довольно коротким 3-6 слов.
"""
print(text)
chatbot_response = chat_with_chatgpt(prompt, model="gpt-4")
print(chatbot_response)

"у меня своя рок группа"  
@  
ЕДИНСТВЕННЫМИ ПАЛОЧКАМИ В ЖИЗНИ БАРАБАНЩИКА БЫЛИ ПАЛОЧКИ ОТ СУШИ  
@  
БАСИСТ ДАЖЕ НЕ ЗНАЕТ ЧТО ОН В ГРУППЕ  
@  
ВОКАЛИСТА НЕТ  
@  
ВСЕ ПЕСНИ ЭТО ГАЧИКАВЕРЫ НА ЕГОРА ЛЕТОВА
Напиши бугурт на тему "безвокальная рок группа":
"Молчание голоса, звук рока."


### export labeled bugurts

In [55]:
import pandas as pd
from datasets import Dataset

dataset = pd.read_csv("./verbalist/datasets/bugurt_thread/bugurt_thread_labeled_v1.csv")
dataset = dataset[~dataset['prompt'].isna()]
dataset = dataset.to_dict('records')
dataset = Dataset.from_list(dataset)
dataset.push_to_hub("dim/bugurt_thread_prompts")

Pushing dataset shards to the dataset hub: 100%|██████████| 1/1 [00:01<00:00,  1.70s/it]


In [56]:
dataset

Dataset({
    features: ['bugurt', 'prompt'],
    num_rows: 108
})